In [41]:
import numpy as np
import pandas as pd
import matplotlib

#Abre os bancos de dados
pop = pd.read_csv('base_csv/pop2020.csv')
io = pd.read_csv('base_csv/caso_full.csv')
io.head()

,city,city_ibge_code,date,epidemiological_week,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
0,São Paulo,3550308.0,2020-02-25,9,12252023.0,False,False,1,0.00816,2020-02-25,0.0,0,1,city,SP,1,0
1,NaN,35.0,2020-02-25,9,45919049.0,False,False,1,0.00218,2020-02-25,0.0,0,1,state,SP,1,0
2,São Paulo,3550308.0,2020-02-26,9,12252023.0,False,False,1,0.00816,2020-02-26,0.0,0,2,city,SP,0,0
3,NaN,35.0,2020-02-26,9,45919049.0,False,False,1,0.00218,2020-02-26,0.0,0,2,state,SP,0,0
4,São Paulo,3550308.0,2020-02-27,9,12252023.0,False,False,1,0.00816,2020-02-27,0.0,0,3,city,SP,0,0


In [36]:
#_________________________Tratamento dos casos por Estados_________________________________________________

In [37]:
#Cria uma coluna com a sigla de cada estado.
states_list = {'ACRE':'AC','ALAGOAS':'AL','AMAPÁ':'AP','AMAZONAS':'AM',
               'BAHIA':'BA','CEARÁ':'CE','ESPÍRITO SANTO':'ES','GOIÁS':'GO',
               'MARANHÃO':'MA','MATO GROSSO':'MT','MATO GROSSO DO SUL':'MS',
               'MINAS GERAIS':'MG','PARÁ':'PA','PARAÍBA':'PB','PARANÁ':'PR',
               'PERNAMBUCO':'PE','PIAUÍ':'PI','RIO DE JANEIRO':'RJ',
               'RIO GRANDE DO NORTE':'RN','RIO GRANDE DO SUL':'RS','RONDÔNIA':'RO',
               'RORAIMA':'RR','SANTA CATARINA':'SC','SÃO PAULO':'SP','SERGIPE':'SE',
               'TOCANTINS':'TO','DISTRITO FEDERAL':'DF'}
pop['state']=pop['ARmaior'].apply(lambda x: states_list[x])





In [38]:
#Cria um DF com o total da população de cada estado.
pop_state = pop[['state','Total']].copy()
pop_state.rename(columns={'Total':'total_inhabitants_state'}, inplace = True)
pop_state = pop_state.groupby(pop_state['state']).sum().reset_index()



In [39]:
#Filtra somente os dados do último dia das semanas epidemiológicas
'''
dates = pd.date_range('20200229',freq='W-SAT',periods=23)

Usei para achar todos os dias e copiei a lista com os dias. Não usei direto, pois estava pegando
os objetos e não consegui trasformar em lista. 
'''
dates = ['2020-02-29', '2020-03-07', '2020-03-14', '2020-03-21',
               '2020-03-28', '2020-04-04', '2020-04-11', '2020-04-18',
               '2020-04-25', '2020-05-02', '2020-05-09', '2020-05-16',
               '2020-05-23', '2020-05-30', '2020-06-06', '2020-06-13',
               '2020-06-20', '2020-06-27', '2020-07-04', '2020-07-11',
               '2020-07-18', '2020-07-25', '2020-08-01']

datesDF = pd.DataFrame(dates,columns=['date'])

select_for_dates = pd.merge(io,datesDF, how = 'inner')


In [53]:
#ATENÇÃO
#Coloca o total da população dos estados na planilha
statesDF = pd.merge(select_for_dates,pop_state, how = 'inner')

#Seleciona apenas os estados
statesDF = statesDF[statesDF['place_type']=='state']


In [54]:
#Encontrar o valor dos novos casoso por semana
statesDF = statesDF.sort_values(by=['state','date']).copy()


last_state = ''
last_num = 0 

    
def get_num(x):
    global last_num, last_state
    if last_state == x['state']:
        current = x['last_available_confirmed']-last_num
        last_num = x['last_available_confirmed']
        return current
    else:
        last_state = x['state']
        last_num = 0
        return get_num(x)



statesDF['new_week_cases'] = statesDF.apply(get_num , axis=1)



In [55]:
#Passa os novos casoso por semana para casos por 100k habitantes
statesDF['new_week_cases_per_100k_inhabitantants'] = statesDF.apply(lambda x:x['new_week_cases']*100000/x['total_inhabitants_state'] , axis=1)

In [56]:
statesDF.head(2)

,city,city_ibge_code,date,epidemiological_week,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths,total_inhabitants_state,new_week_cases,new_week_cases_per_100k_inhabitantants
53780,NaN,12.0,2020-03-21,12,881935.0,False,False,11,1.24726,2020-03-21,0.0,0,5,state,AC,4,0,894469.979591,11,1.229779
53782,NaN,12.0,2020-03-28,13,881935.0,False,False,25,2.83468,2020-03-28,0.0,0,12,state,AC,0,0,894469.979591,14,1.565173


In [68]:
#Indicador de Letalidade
def lethality(x):
    if x['last_available_confirmed']==0:
        return 0
    else:
        return x['last_available_deaths']/x['last_available_confirmed']
statesDF['lethality'] = statesDF.apply(lethality, axis=1)
statesDF

,city,city_ibge_code,date,epidemiological_week,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,...,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths,total_inhabitants_state,new_week_cases,new_week_cases_per_100k_inhabitantants,lethality
53780,NaN,12.0,2020-03-21,12,881935.0,False,False,11,1.24726,2020-03-21,...,0,5,state,AC,4,0,8.944700e+05,11,1.229779,0.000000
53782,NaN,12.0,2020-03-28,13,881935.0,False,False,25,2.83468,2020-03-28,...,0,12,state,AC,0,0,8.944700e+05,14,1.565173,0.000000
53786,NaN,12.0,2020-04-04,14,881935.0,False,False,46,5.21580,2020-04-04,...,0,19,state,AC,0,0,8.944700e+05,21,2.347759,0.000000
53792,NaN,12.0,2020-04-11,15,881935.0,False,False,72,8.16387,2020-04-11,...,2,26,state,AC,2,0,8.944700e+05,26,2.906749,0.027778
53799,NaN,12.0,2020-04-18,16,881935.0,False,False,142,16.10096,2020-04-18,...,6,33,state,AC,7,1,8.944700e+05,70,7.825864,0.042254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68640,NaN,17.0,2020-07-04,27,1572866.0,False,False,12476,793.20171,2020-07-04,...,220,109,state,TO,194,5,1.590248e+06,2248,141.361598,0.017634
68780,NaN,17.0,2020-07-11,28,1572866.0,False,False,15132,962.06543,2020-07-11,...,255,116,state,TO,193,4,1.590248e+06,2656,167.017974,0.016852
68920,NaN,17.0,2020-07-18,29,1572866.0,False,False,17773,1129.97547,2020-07-18,...,294,123,state,TO,564,6,1.590248e+06,2641,166.074725,0.016542
69060,NaN,17.0,2020-07-25,30,1572866.0,False,False,21767,1383.90683,2020-07-25,...,346,130,state,TO,847,6,1.590248e+06,3994,251.155794,0.015896


In [58]:
#Filtra somente as colunas de interesse
states_incidence = statesDF[['state','city_ibge_code','date','epidemiological_week','total_inhabitants_state','new_week_cases','new_week_cases_per_100k_inhabitantants']].copy()
states_incidence.rename(columns={'city_ibge_code':'ibge_code','date':'week'}, inplace = True)


In [59]:
#Inserindo coordenadas geogŕaficas dos estados
estados = pd.read_csv('base_csv/estados.csv')
estados = estados[['codigo_uf','latitude','longitude']].copy()
states_incidence = pd.merge(states_incidence,estados,
                          how = 'inner', left_on=['ibge_code'],
                          right_on=['codigo_uf'])
states_incidence = states_incidence.drop(columns=['codigo_uf'])


In [60]:
#Agrupa por macro regiões
def region(x):
    regions = {
        'norte':['AM','RR','AP','PA','TO','RO','AC'],
        'nordeste':['MA','PI','CE','RN','PE','PB','SE','AL','BA'],
        'centro-oeste':['MT','MS','GO','DF'],
        'sudeste':['MG','SP','RJ','ES'],
        'sul':['PR','RS','SC']       
    }
    for k,v in regions.items():
        if x['state'] in v:
            return k
    
states_incidence['region'] = states_incidence.apply(region,axis=1)


In [66]:
#Salvando em csv
states_incidence.to_csv('resultados/states_incidence.csv')
states_incidence[1:300]

,state,ibge_code,week,epidemiological_week,total_inhabitants_state,new_week_cases,new_week_cases_per_100k_inhabitantants,latitude,longitude,region
1,AC,12.0,2020-03-28,13,8.944700e+05,14,1.565173,-8.77,-70.55,norte
2,AC,12.0,2020-04-04,14,8.944700e+05,21,2.347759,-8.77,-70.55,norte
3,AC,12.0,2020-04-11,15,8.944700e+05,26,2.906749,-8.77,-70.55,norte
4,AC,12.0,2020-04-18,16,8.944700e+05,70,7.825864,-8.77,-70.55,norte
5,AC,12.0,2020-04-25,17,8.944700e+05,116,12.968574,-8.77,-70.55,norte
...,...,...,...,...,...,...,...,...,...,...
295,PB,25.0,2020-04-11,15,4.039277e+06,67,1.658713,-7.28,-36.72,nordeste
296,PB,25.0,2020-04-18,16,4.039277e+06,135,3.342182,-7.28,-36.72,nordeste
297,PB,25.0,2020-04-25,17,4.039277e+06,263,6.511066,-7.28,-36.72,nordeste
298,PB,25.0,2020-05-02,18,4.039277e+06,670,16.587127,-7.28,-36.72,nordeste


In [62]:
#Agrupa os dados por semana epidemiológica, para todo Brasil
states_incidence_brazil = states_incidence.groupby(['epidemiological_week']).sum().reset_index()
states_incidence_brazil

,epidemiological_week,ibge_code,total_inhabitants_state,new_week_cases,new_week_cases_per_100k_inhabitantants,latitude,longitude
0,9,35.0,4.628933e+07,2,0.004321,-22.19,-48.79
1,10,182.0,8.570254e+07,17,0.106041,-92.75,-221.36
2,11,584.0,1.741814e+08,117,1.047637,-274.98,-776.68
3,12,786.0,2.117557e+08,996,12.288473,-334.29,-1307.08
4,13,786.0,2.117557e+08,2821,29.991042,-334.29,-1307.08
5,14,786.0,2.117557e+08,6428,60.195930,-334.29,-1307.08
6,15,786.0,2.117557e+08,10610,119.569820,-334.29,-1307.08
7,16,786.0,2.117557e+08,16184,209.629809,-334.29,-1307.08
8,17,786.0,2.117557e+08,22331,313.286207,-334.29,-1307.08
9,18,786.0,2.117557e+08,37924,543.465410,-334.29,-1307.08


In [63]:
#Gráfico Brasil
import plotly.express as px

fig = px.line(states_incidence_brazil,x='epidemiological_week',
              y='new_week_cases_per_100k_inhabitantants',title = 'Brazilian incidence')
fig.show()

In [31]:
#Agrupa os dados por regiões e por semana epidemiológica
states_incidence_grouped = states_incidence.groupby(['region','epidemiological_week']).sum().reset_index()


In [32]:
#Gráficos por Regiões
import plotly.express as px

fig = px.line(states_incidence_grouped,x='epidemiological_week',
              y='new_week_cases_per_100k_inhabitantants',
              title = 'Brazilian incidence per region',color='region')
fig.show()


In [64]:
#Gráficos por estados Retirados por não serem requisito do projeto
''''for i in range(len(pop_state)):
    one_state_incidence = states_incidence.loc[lambda df: df['state']== pop_state['state'][i]]

    fig =one_state_incidence.plot(y='new_week_cases_per_100k_inhabitantants',x='week',
                                  ylim=(0,1000), 
                                  figsize = (14,8),
                              title = (pop_state['state'][i] +' new_week_cases_per_100k_inhabitantants')).get_figure()
    fig.savefig(('graficos/'+pop_state['state'][i] +' new_week_cases_per_100k_inhabitantants.pdf'))'''

"'for i in range(len(pop_state)):\n    one_state_incidence = states_incidence.loc[lambda df: df['state']== pop_state['state'][i]]\n\n    fig =one_state_incidence.plot(y='new_week_cases_per_100k_inhabitantants',x='week',\n                                  ylim=(0,1000), \n                                  figsize = (14,8),\n                              title = (pop_state['state'][i] +' new_week_cases_per_100k_inhabitantants')).get_figure()\n    fig.savefig(('graficos/'+pop_state['state'][i] +' new_week_cases_per_100k_inhabitantants.pdf'))"

In [13]:
#_________________________Tratamento dos casos por Cidades_________________________________________________

In [14]:
#Abre os bancos de dados
pop = pd.read_csv('base_csv/pop2020.csv')
io = pd.read_csv('base_csv/caso_full.csv')

In [15]:
#Renomeia as colunas do DF pop e cria um novo DF só com os dados de interesse
pop.rename(columns={'Total':'total_inhabitants', 'Armenor':'city_ibge_code'}, inplace = True)
pop_cities = pop[['city_ibge_code','total_inhabitants']].copy()


In [16]:
#Filtra somente os dados do último dia das semanas epidemiológicas
'''
dates = pd.date_range('20200229',freq='W-SAT',periods=25)

Usei para achar todos os dias e copiei a lista com os dias. Não usei direto, pois estava pegando
os objetos e não consegui trasformar em lista, mas deixei aqui para ficar fácil de alterar as semanas. 
'''
dates = ['2020-02-29', '2020-03-07', '2020-03-14', '2020-03-21',
               '2020-03-28', '2020-04-04', '2020-04-11', '2020-04-18',
               '2020-04-25', '2020-05-02', '2020-05-09', '2020-05-16',
               '2020-05-23', '2020-05-30', '2020-06-06', '2020-06-13',
               '2020-06-20', '2020-06-27', '2020-07-04', '2020-07-11',
               '2020-07-18', '2020-07-25', '2020-08-01', '2020-08-08',
               '2020-08-15']

datesDF = pd.DataFrame(dates,columns=['date'])

select_for_dates = pd.merge(io,datesDF, how = 'inner')


In [17]:
#Coloca o total da população dos estados na planilha
citiesDF = pd.merge(select_for_dates,pop_cities, how = 'inner')


In [18]:
#Encontrar o valor dos novos casoso por semana
citiesDF = citiesDF.sort_values(by=['city_ibge_code','date']).copy()

last_city = 0
last_num = 0 


    
def get_num(x):
    global last_num, last_city
    if last_city == x['city_ibge_code']:
        current = x['last_available_confirmed']-last_num
        last_num = x['last_available_confirmed']
        return current
    else:
        last_city = x['city_ibge_code']
        last_num = 0
        return get_num(x)



citiesDF['new_week_cases'] = citiesDF.apply(get_num , axis=1)



In [19]:
#Passa os novos casoso por semana para casos por 100k habitantes
citiesDF['new_week_cases_per_100k_inhabitantants'] = citiesDF.apply(lambda x:x['new_week_cases']*100000/x['total_inhabitants'] , axis=1)


In [20]:
#Filtra somente as colunas de interesse
cities_incidence = citiesDF[['city','state','city_ibge_code','date','total_inhabitants','new_week_cases','new_week_cases_per_100k_inhabitantants']].copy()
cities_incidence.rename(columns={'date':'week'}, inplace = True)

In [21]:
#Inserindo coordenadas geogŕaficas nos municípios
municipios = pd.read_csv('base_csv/municipios.csv')
municipios = municipios[['codigo_ibge','latitude','longitude']].copy()

cities_incidence = pd.merge(cities_incidence,municipios,
                          how = 'inner', left_on=['city_ibge_code'],
                          right_on=['codigo_ibge'])
cities_incidence = cities_incidence.drop(columns=['codigo_ibge'])



In [22]:
#Salvando em csv

cities_incidence.to_csv('resultados/cities_incidence.csv')
cities_incidence

,city,state,city_ibge_code,week,total_inhabitants,new_week_cases,new_week_cases_per_100k_inhabitantants,latitude,longitude
0,Alta Floresta D'Oeste,RO,1100015.0,2020-05-02,2.208105e+04,1,4.528770,-11.9283,-61.9953
1,Alta Floresta D'Oeste,RO,1100015.0,2020-05-09,2.208105e+04,0,0.000000,-11.9283,-61.9953
2,Alta Floresta D'Oeste,RO,1100015.0,2020-05-16,2.208105e+04,0,0.000000,-11.9283,-61.9953
3,Alta Floresta D'Oeste,RO,1100015.0,2020-05-23,2.208105e+04,3,13.586309,-11.9283,-61.9953
4,Alta Floresta D'Oeste,RO,1100015.0,2020-05-30,2.208105e+04,-1,-4.528770,-11.9283,-61.9953
...,...,...,...,...,...,...,...,...,...
79342,Brasília,DF,5300108.0,2020-07-18,3.052546e+06,10747,352.066767,-15.7795,-47.9297
79343,Brasília,DF,5300108.0,2020-07-25,3.052546e+06,11488,376.341585,-15.7795,-47.9297
79344,Brasília,DF,5300108.0,2020-08-01,3.052546e+06,11930,390.821301,-15.7795,-47.9297
79345,Brasília,DF,5300108.0,2020-08-08,3.052546e+06,12415,406.709678,-15.7795,-47.9297
